# 2pt

In [43]:
import h5py,os,re,click
import numpy as np
from itertools import permutations
from sympy.combinatorics import Permutation

ens='cE211.044.112'

path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

#!============== mom opearations ==============#
if True:
    def mom2msq(mom):
        assert(len(mom)==3)
        return mom[0]**2+mom[1]**2+mom[2]**2
    def getSortkey_mom(mom): # @mom2standard
        msq=mom2msq(mom)
        return (msq,-mom[2],-mom[1],-mom[0])
    def mom2standard(mom): # @getSortkey_mom
        return sorted([abs(mom[0]),abs(mom[1]),abs(mom[2])]) 
    
    elements_rot48=[(sx,sy,sz,xyz) for sx in [1,-1] for sy in [1,-1] for sz in [1,-1] for xyz in [(0,1,2),(0,2,1),(1,0,2),(1,2,0),(2,0,1),(2,1,0)]]
    def rotate_mom(e,mom):
        if len(mom)!=3:
            return rotate_mom(e,mom[:3])+rotate_mom(e,mom[3:])
        sx,sy,sz,xyz=e; ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
        return [sx*mom[iix],sy*mom[iiy],sz*mom[iiz]]
    def mom2moms(mom):
        return [list(mom) for mom in {tuple(rotate_mom(e,mom)) for e in elements_rot48}] 
    
max_mom2=64
range_xyz=range(-int(np.sqrt(max_mom2))-1,int(np.sqrt(max_mom2))+2)
moms=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2]       
moms=list(set([tuple(mom2standard(mom)) for mom in moms]))
moms=[list(mom) for mom in moms]; 
moms=sorted(moms,key=getSortkey_mom)
moms_target=moms

def extractFile(paths):
    Nsrcs=[]
    for ipath,path in enumerate(paths):
        with h5py.File(path) as f:
            Nsrcs.append(len(f['srcs']))
    
    dat=0
    for ipath,path in enumerate(paths):
        with h5py.File(path) as f:
            t1=f['data/N1_N1'][:]; t2=f['data/N2_N2'][:]
            t=(t1+t2)/2
            t=np.mean(t,axis=-1)

            t1=f['data_bw/N1_N1'][:]; t2=f['data_bw/N2_N2'][:]
            t_bw=(t1+t2)/2
            t_bw=np.mean(t_bw,axis=-1)
            
            t_bw=np.flip(t_bw,axis=0)
            # t_bw=t_bw[:,inds_negmom]
            t[1:] = (t[1:] + (-1)*t_bw[:])/2
            
            moms=f['moms'][:]
            dic={}
            for i,m in enumerate(moms):
                dic[tuple(m)]=i

            t=np.transpose([np.mean(t[:,[dic[tuple(m)] for m in mom2moms(mom)]],axis=1) for mom in moms_target],[1,0])
            
            weight=Nsrcs[ipath]/np.sum(Nsrcs)
            dat += np.real(t*weight)
    return dat
    
data=[]
for icfg,cfg in enumerate(cfgs):
    print(f'{icfg}/{len(cfgs)}',end='        \r')
    
    path=f'/p/project1/ngff/li47/code/projectData/05_moments/{ens}/data_N_fullmom/{cfg}/'
    data.append(extractFile([f'{path}/{file}' for file in os.listdir(path) if ('run4' not in file) and ('run5' not in file)]))

data=np.array(data)

path=f'/p/project1/ngff/li47/code/projectData/05_moments/{ens}/data_merge/disc_2pt.h5'
with h5py.File(path,'w') as f:
    f.create_dataset('notes',data=['cfg,time,mom'])
    f.create_dataset('cfgs',data=cfgs)
    f.create_dataset('moms',data=moms_target)
    
    f.create_dataset('data/N_N',data=data)

print(data.shape)

(502, 57, 80)  


In [42]:
path1='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_merge/disc_2pt.h5'
path2='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_merge/disc_2pt_backup.h5'
with h5py.File(path1) as f1, h5py.File(path2) as f2:
    cfgs=[cfg.decode() for cfg in f1['cfgs'][:]]
    i=cfgs.index('a1084')
    print(i)
    t1=f1['data/N_N'][:]; t2=f2['data/N_N'][:]
    dt=np.abs(t1-t2)
    temp=dt/t1
    print(temp[i,55,0])
    print(t2[i,54])

71
nan
[ 2.46107790e-15 -3.81415856e-16  3.49291342e-17  3.57388407e-16
  8.53975927e-16  7.80924999e-16  4.79007581e-16  5.33046720e-16
 -1.44957882e-16  3.18241696e-16  1.03906955e-16  1.42702730e-16
  5.77656104e-16 -3.12922899e-17 -1.82700774e-16 -4.36704004e-16
  1.29725935e-16 -1.09221548e-16  4.34239408e-17 -2.08695079e-16
 -5.53661195e-16  2.19714450e-16 -1.12713965e-16 -2.13560946e-16
 -1.12410778e-16  1.84359719e-17 -9.03027616e-17  9.46252541e-17
 -6.57670394e-18 -1.48712414e-17 -2.79078049e-17 -4.16813053e-17
 -1.15549787e-16 -5.00490726e-17 -7.93723267e-17 -7.16293254e-17
 -6.32581620e-17 -2.14147059e-17 -3.80404495e-17 -3.56705479e-17
  9.84229745e-17 -6.63137995e-17 -1.03445517e-18 -5.49388277e-17
 -8.83554253e-17 -6.66834383e-17 -7.08315514e-17  2.09383838e-17
  4.10360217e-18 -7.75199047e-17  3.71870619e-17 -3.88928711e-17
 -6.46847774e-17 -6.64997030e-17 -1.98417746e-17  8.36635383e-17
 -8.38053486e-18  7.28679059e-18  1.87653486e-17 -1.06752616e-17
  1.56068859e-17  

In [18]:
path1='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_merge/disc_2pt.h5'
path2='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_merge/disc_2pt_backup.h5'
with h5py.File(path1) as f1, h5py.File(path2) as f2:
    cfgs=f1['cfgs'][:]
    t1=f1['data/N_N'][:]; t2=f2['data/N_N'][:]
    dt=np.abs(t1-t2)
    print(dt.shape)
    dt=np.sum(np.sum(dt,axis=1),axis=1)
    for i in range(len(dt)):
        if dt[i]!=0:
            print(dt[i],cfgs[i])

(516, 57, 80)
9.273425173223602e-11 b'a1004'
7.646136923794442e-11 b'a1012'
1.8425699480117928e-10 b'a1020'
1.0746743752434451e-10 b'a1028'
1.4512983750772364e-10 b'a1036'
3.4977430821826045e-10 b'a1044'
1.4258298334704638e-10 b'a1052'
1.0814061457509883e-10 b'a1060'
3.10539464143645e-10 b'a1068'
3.069528058612841e-10 b'a1076'
nan b'a1084'
8.992827472519469e-11 b'a1092'
2.0890772631830776e-10 b'a1100'
9.70893905088523e-11 b'a1108'
4.34860424104679e-10 b'a1116'
4.0012981392538195e-10 b'a1124'
1.0073325337151087e-10 b'a1132'
2.3774357833776957e-10 b'a1140'
1.4398083049253604e-10 b'a1148'
8.402996268313039e-11 b'a1156'
8.636577444121025e-11 b'a1164'
7.150897731803244e-11 b'a1172'
1.2277031304512847e-10 b'a1180'
1.5992544911145234e-10 b'a1188'
1.4954005762385053e-10 b'a1196'
2.5973303734055776e-10 b'a1204'
1.8727740422332892e-10 b'a1212'
7.684019666616885e-11 b'a1220'
9.717008945173273e-11 b'a1228'
nan b'a1236'
2.1245540955365226e-10 b'a1244'
1.301239810420259e-10 b'a1252'
1.45774535859077

In [75]:
path='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_N_fullmom/d0352/N.h5_Nrun2_127'
with h5py.File(path) as f:
    t=f['data/N2_N2']
    print(t.shape)
    print(t[:16,0,0])
    print(t[6,0,0])

(57, 2109, 2)
[ 1.0175419e-10-5.1544950e-13j  2.5773803e-10-1.0312250e-12j
  1.6366851e-10-1.3036139e-12j  1.0939998e-10+1.4737686e-12j
  4.2120382e-06+1.7305374e+00j            inf          +nanj
            inf          +nanj  5.4853951e+30+6.2110817e-01j
  2.3872912e-11-1.7985359e+00j -1.1285164e+22+4.4034547e-01j
  9.1660281e+35+6.5844530e-01j  2.4737799e-11+1.7469832e+00j
  8.5907576e+15-1.4577904e+00j -4.8055490e+14+1.8628734e+00j
 -3.5569411e+30-4.0908760e-01j -4.3823585e+09+1.6170362e+00j]
(inf+nanj)


In [85]:
import tarfile
path='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_pre/Nrun2_127_arch/0352_r3.tar'
with tarfile.open(path) as tar:
    for member in tar.getmembers():
        if not member.name.endswith('.h5'):
            continue
        if not ('twop' in member.name):
            continue
        if not ('sx79sy20sz87st200' in member.name):
            continue
        fileobj = tar.extractfile(member)
        with h5py.File(fileobj) as f:
            srcs_old=list(f.keys())
            src_old=srcs_old[0]
            t=f[src_old]['baryons/nucl_nucl/twop_baryon_2']
            print(src_old,t[6,0,0,0])
            t=f[src_old]['baryons/nucl_nucl/twop_baryon_1']
            print(src_old,t[6,0,0,0])
        # break

sx79sy20sz87st200 inf
sx79sy20sz87st200 0.0


# jvev

In [ ]:
import h5py,os,re,click
import numpy as np
from itertools import permutations
from sympy.combinatorics import Permutation

ens='cB211.072.64'

jqs=['j+','js','jc'] # disc
stouts=range(40+1) # gluon

path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
inds_trace=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]

xyzt2xyz0=lambda x: x if x!='t' else '0'
t=[insert for insert in inserts]; inserts_key=[f'=der:g{xyzt2xyz0(insert[1])}D{xyzt2xyz0(insert[0])}:sym=' for insert in t]

def extractVEV(basepath):
    j2data={}
    
    txyz=['t','x','y','z']
    Dmus=['d3','d0','d1','d2']
    gnus=['gt','gx','gy','gz']
    
    path=f'{basepath}/j.h5'
    with h5py.File(path) as f:
        for j in jqs:
            gms=[gm.decode() for gm in f['inserts'][:]]
            moms_old=f['moms'][:]
            dic={}
            for i,m in enumerate(moms_old):
                dic[tuple(m)]=i
            imom=dic[(0,0,0)]
            
            t=np.array([[f[f'data/{j};{Dmu}'][:,:,gms.index(gnu)] for Dmu in Dmus] for gnu in gnus])
            t=t[:,:,:,imom]
            t=(t+np.transpose(t,[1,0,2]))/2
            t=t - np.eye(4)[:,:,None]*np.trace(t,axis1=0,axis2=1)[None,None,:]/4
            t=np.transpose([t[txyz.index(m),txyz.index(n)] for m,n in inserts],[1,0])
            
            j2data[f'{j};disc']=np.mean(t,axis=0)

    path=f'{basepath}/jg.h5'
    with h5py.File(path) as f:
        gms=[gm.decode() for gm in f['inserts'][:]]
        moms_old=f['moms'][:]
        dic={}
        for i,m in enumerate(moms_old):
            dic[tuple(m)]=i
        imom=dic[(0,0,0)]
        for stout in stouts:
            j=f'jg;stout{stout}'
            
            t=f[f'data/{j}'][:]
            t=t[:,imom]
            j2data[j]=np.mean(t,axis=0)
            
    return j2data

js=[f'{j};disc' for j in jqs] + [f'jg;stout{stout}' for stout in stouts]
j2data={j:[] for j in js}
for icfg,cfg in enumerate(cfgs):
    print(f'{icfg}/{len(cfgs)}',end='        \r')
    
    basepath=f'/p/project1/ngff/li47/code/projectData/02_discNJN_1D/{ens}/data_post_hold/{cfg}/'
    j2dat=extractVEV(basepath)
    
    for j in js:
        j2data[j].append(j2dat[j])
    # break

path=f'/p/project1/ngff/li47/code/projectData/05_moments/{ens}/data_merge/disc_jvev.h5'
with h5py.File(path,'w') as f:
    for j in j2data.keys():
        f.create_dataset(j,data=j2data[j])